# Day 17

In [4]:
def solve_chronospatial_computer(initial_a, initial_b, initial_c, program):
    registers = {'A': initial_a, 'B': initial_b, 'C': initial_c}
    instruction_pointer = 0
    output = []

    while instruction_pointer < len(program):
        opcode = program[instruction_pointer]
        operand = program[instruction_pointer + 1] if instruction_pointer + 1 < len(program) else None
        
        #print(f"IP: {instruction_pointer}, Opcode: {opcode}, Operand: {operand}, Registers: {registers}")

        if opcode == 0:  # adv
            divisor = 2 ** get_combo_operand_value(operand, registers)
            registers['A'] //= divisor
            instruction_pointer += 2
        elif opcode == 1:  # bxl
            registers['B'] ^= operand
            instruction_pointer += 2
        elif opcode == 2:  # bst
            registers['B'] = get_combo_operand_value(operand, registers) % 8
            instruction_pointer += 2
        elif opcode == 3:  # jnz
            if registers['A'] != 0:
                instruction_pointer = operand
            else:
                instruction_pointer += 2
        elif opcode == 4:  # bxc
            registers['B'] ^= registers['C']
            instruction_pointer += 2
        elif opcode == 5:  # out
            output.append(str(get_combo_operand_value(operand, registers) % 8))
            instruction_pointer += 2
        elif opcode == 6:  # bdv
            divisor = 2 ** get_combo_operand_value(operand, registers)
            registers['B'] //= divisor
            instruction_pointer += 2
        elif opcode == 7:  # cdv
            divisor = 2 ** get_combo_operand_value(operand, registers)
            registers['C'] //= divisor
            instruction_pointer += 2
        else:
            break  # Halt if invalid opcode

    return ",".join(output)


def get_combo_operand_value(operand, registers):
    if 0 <= operand <= 3:
        return operand
    elif operand == 4:
        return registers['A']
    elif operand == 5:
        return registers['B']
    elif operand == 6:
        return registers['C']
    else:
       return None # Invalid combo operand

# --- Puzzle Input:
# Register A: 61156655
# Register B: 0
# Register C: 0
# Program: 2,4,1,5,7,5,4,3,1,6,0,3,5,5,3,0

program_input = [2,4,1,5,7,5,4,3,1,6,0,3,5,5,3,0]
output_puzzle = solve_chronospatial_computer(61156655, 0, 0, program_input)
print(f"Puzzle Output: {output_puzzle}")

Puzzle Output: 4,6,7,5,1,1,2,6,0


In [6]:
def execute_instruction(program, pointer, registers):
    opcode = program[pointer]

    if opcode == 0:  # adv
        operand = program[pointer + 1]
        denominator = 2** (operand if operand < 4 else registers[(operand - 4)])
        registers['A'] //= denominator
        pointer += 2
    elif opcode == 1: # bxl
        operand = program[pointer + 1]
        registers['B'] ^= operand
        pointer += 2
    elif opcode == 2:  # bst
        operand = program[pointer + 1]
        registers['B'] = operand % 8
        pointer += 2
    elif opcode == 3:  # jnz
        operand = program[pointer + 1]
        if registers['A']:
            pointer = operand
        else:
            pointer += 2
    elif opcode == 4: # bxc
        operand = program[pointer + 1]
        registers['B'] ^= registers['C']
        pointer += 2  
    elif opcode == 5:  # out
        # Correction: Print the value of register 'B' modulo 8
        print(registers['B'] % 8, end=",") 
        pointer += 2
    elif opcode == 6:  # bdv
        operand = program[pointer + 1]
        denominator = 2**(operand if operand < 4 else registers[(operand - 4)])
        registers['B'] //= denominator
        pointer += 2
    elif opcode == 7: # cdv
        operand = program[pointer + 1]
        denominator = 2**(operand if operand < 4 else registers[(operand - 4)])
        registers['C'] //= denominator
        pointer += 2

    return pointer



# Example program and register initialization
program = [0, 1, 5, 4, 3, 0] # Replace with the actual program from input
registers = {'A': 729, 'B': 0, 'C': 0}  # Replace with initial register values

pointer = 0
while pointer < len(program):
    pointer = execute_instruction(program, pointer, registers)




0,0,0,0,0,0,0,0,0,0,

In [8]:
def run_program(registers, program):
    # Unpack registers
    A, B, C = registers
    ip = 0  # Instruction pointer
    output = []  # Store output values

    # Helper function to resolve a combo operand
    def resolve_combo(operand):
        if operand <= 3:
            return operand  # Literal value 0-3
        elif operand == 4:
            return A
        elif operand == 5:
            return B
        elif operand == 6:
            return C
        else:
            raise ValueError("Invalid combo operand: 7 is reserved.")

    # Execution loop
    while ip < len(program):
        opcode = program[ip]
        operand = program[ip + 1]

        if opcode == 0:  # adv: divide A by 2^operand (combo operand)
            denom = 2 ** resolve_combo(operand)
            A //= denom

        elif opcode == 1:  # bxl: B ^= operand (literal operand)
            B ^= operand

        elif opcode == 2:  # bst: B = operand % 8 (combo operand)
            B = resolve_combo(operand) % 8

        elif opcode == 3:  # jnz: if A != 0, jump to operand (literal operand)
            if A != 0:
                ip = operand
                continue  # Skip incrementing the IP

        elif opcode == 4:  # bxc: B ^= C
            B ^= C

        elif opcode == 5:  # out: output operand % 8 (combo operand)
            output.append(resolve_combo(operand) % 8)

        elif opcode == 6:  # bdv: B = A // 2^operand (combo operand)
            denom = 2 ** resolve_combo(operand)
            B = A // denom

        elif opcode == 7:  # cdv: C = A // 2^operand (combo operand)
            denom = 2 ** resolve_combo(operand)
            C = A // denom

        else:
            raise ValueError(f"Invalid opcode: {opcode}")

        ip += 2  # Move to the next instruction

    return ",".join(map(str, output))


# Sample Input
initial_registers = (61156655, 0, 0)
program = [2,4,1,5,7,5,4,3,1,6,0,3,5,5,3,0]

# Register A: 61156655
# Register B: 0
# Register C: 0

# Program: 2,4,1,5,7,5,4,3,1,6,0,3,5,5,3,0


# Run the program
result = run_program(initial_registers, program)
print(result)


7,3,5,7,5,7,4,3,0


In [ ]:
def run_program(registers, program):
    # Unpack registers
    A, B, C = registers
    ip = 0  # Instruction pointer
    output = []  # Store output values

    # Helper function to resolve a combo operand
    def resolve_combo(operand):
        if operand <= 3:
            return operand  # Literal value 0-3
        elif operand == 4:
            return A
        elif operand == 5:
            return B
        elif operand == 6:
            return C
        else:
            raise ValueError("Invalid combo operand: 7 is reserved.")

    # Execution loop
    while ip < len(program):
        opcode = program[ip]
        operand = program[ip + 1]

        if opcode == 0:  # adv: divide A by 2^operand (combo operand)
            denom = 2 ** resolve_combo(operand)
            A //= denom

        elif opcode == 1:  # bxl: B ^= operand (literal operand)
            B ^= operand

        elif opcode == 2:  # bst: B = operand % 8 (combo operand)
            B = resolve_combo(operand) % 8

        elif opcode == 3:  # jnz: if A != 0, jump to operand (literal operand)
            if A != 0:
                ip = operand
                continue  # Skip incrementing the IP

        elif opcode == 4:  # bxc: B ^= C
            B ^= C

        elif opcode == 5:  # out: output operand % 8 (combo operand)
            output.append(resolve_combo(operand) % 8)

        elif opcode == 6:  # bdv: B = A // 2^operand (combo operand)
            denom = 2 ** resolve_combo(operand)
            B = A // denom

        elif opcode == 7:  # cdv: C = A // 2^operand (combo operand)
            denom = 2 ** resolve_combo(operand)
            C = A // denom

        else:
            raise ValueError(f"Invalid opcode: {opcode}")

        ip += 2  # Move to the next instruction

    return output


def find_initial_value_for_output(program):
    # Start searching for the lowest valid A
    initial_A = 1  # Start from the lowest positive value
    while True:
        # Simulate the program with the current value of A
        output = run_program((initial_A, 0, 0), program)
        # Check if the output matches the program itself
        if output == program:
            return initial_A
        # Increment A and try again
        initial_A += 1


# Input for part two
initial_registers = (61156655, 0, 0)
program = [2, 4, 1, 5, 7, 5, 4, 3, 1, 6, 0, 3, 5, 5, 3, 0]

# Find the smallest initial A that reproduces the program
result = find_initial_value_for_output(program)
result


In [12]:
import copy

with open("Input/InputDay17P1.txt") as f:
    registers, programs = f.read().split("\n\n")

registers_original = {
    line.split(": ")[0].split()[1]: int(line.split(": ")[1])
    for line in registers.splitlines()
}
programs = [int(num) for num in programs.replace("\n", "").split()[1].split(",")]


def combo(num, registers):
    if num <= 3:
        return num
    elif num == 4:
        return registers["A"]
    elif num == 5:
        return registers["B"]
    elif num == 6:
        return registers["C"]


def adv(num, registers, instr, outputs):
    registers["A"] = registers["A"] // (2 ** combo(num, registers))
    return registers, instr + 2, outputs


def bxl(num, registers, instr, outputs):
    registers["B"] = registers["B"] ^ num
    return registers, instr + 2, outputs


def bst(num, registers, instr, outputs):
    registers["B"] = combo(num, registers) % 8
    return registers, instr + 2, outputs


def jnz(num, registers, instr, outputs):
    if registers["A"] == 0:
        return registers, instr + 2, outputs
    else:
        return registers, num, outputs


def bxc(num, registers, instr, outputs):
    registers["B"] = registers["B"] ^ registers["C"]
    return registers, instr + 2, outputs


def out(num, registers, instr, outputs):
    outputs.append(combo(num, registers) % 8)
    return registers, instr + 2, outputs


def bdv(num, registers, instr, outputs):
    registers["B"] = registers["A"] // (2 ** combo(num, registers))
    return registers, instr + 2, outputs


def cdv(num, registers, instr, outputs):
    registers["C"] = registers["A"] // (2 ** combo(num, registers))
    return registers, instr + 2, outputs


def get_output(a):
    outputs = []
    registers = copy.deepcopy(registers_original)
    registers["A"] = a
    length = len(programs)
    instr = 0
    while instr in range(length):
        opcode = programs[instr]
        function = [adv, bxl, bst, jnz, bxc, out, bdv, cdv][opcode]
        num = programs[instr + 1]
        registers, instr, outputs = function(num, registers, instr, outputs)
    return outputs


valid = [0]
for length in range(1, len(programs) + 1):
    oldValid = valid
    valid = []
    for num in oldValid:
        for offset in range(8):
            newNum = 8 * num + offset
            if get_output(newNum) == programs[-length:]:
                valid.append(newNum)

answer = min(valid)
print(answer)

105734774294938
